In [9]:
import importlib
import sys

#from src.ui import MatplotlibGUI
from src.cell import GridCoordinates

sys.path.append("src")

import board
import cell
import piece
import game_engine
import ui
importlib.reload(board)
importlib.reload(cell)
importlib.reload(piece)
importlib.reload(game_engine)
importlib.reload(ui)
from board import Board
from cell import Cell
from piece import Piece, Ant, Queen, Spider, Grasshopper, Beetle
from game_engine import Game
from ui import UI, MatplotlibGUI
from cell import Cell, GridCoordinates
from move import Move


board_halfwidth_t = 7
cell_size_t = 1.3
canvas_size_x_t = 40
canvas_size_y_t = 40

g = Game(board_halfwidth_t)
ui = MatplotlibGUI(g, cell_size=cell_size_t, canvas_size_x=canvas_size_x_t, canvas_size_y=canvas_size_y_t)

c1 = GridCoordinates(0,0)
c2 = GridCoordinates(1,+0)
c3 = GridCoordinates(2,0)
c4 = GridCoordinates(1,-1)
c5 = GridCoordinates(2,-2)
c6 = GridCoordinates(2,-1)
c7 = GridCoordinates(4,0)
c8 = GridCoordinates(2,-4)
c9 = GridCoordinates(0,1)
c10 = GridCoordinates(0,4)

ui.place_piece(c1,g.piece_bank_white["queen"])
ui.place_piece(c2,g.piece_bank_white["ant2"])
ui.place_piece(c3,g.piece_bank_white["ant1"])
ui.place_piece(c4,g.piece_bank_black["queen"])
ui.place_piece(c5,g.piece_bank_black["ladybug"])
ui.place_piece(c6,g.piece_bank_black["grasshopper1"])
ui.place_piece(c7,g.piece_bank_black["grasshopper2"])
ui.place_piece(c8,g.piece_bank_black["ladybug"])

ui.move_piece(c3,c9,g.piece_bank_white["ant1"])
# ui.draw_outer_border()
#
ui.draw_board(show_grid=False, show_coords=False)
ui.draw_stats()
ui.draw_piece_banks()
ui.show_canvas()
print(f"Size of game object: {sys.getsizeof(g.cells)}")

#print(f"Valid game state: {g.is_valid_state()}")
#TODO This is not correct - problem with piece comparison (probably comparing different objects)
print(f"Move is legal: {g.is_move_legal(Move(c3,c6,g.piece_bank_white['ant1']))}")




Piece white queen is now on the board at coord(0, 0, 0).
Piece white ant is now on the board at coord(1, 0, -1).
Piece white ant is now on the board at coord(2, 0, -2).
Can place piece only at cells that don't touch other player's pieces
[<cell.Cell object at 0x106035c90>, <cell.Cell object at 0x1037bb490>, <cell.Cell object at 0x1037b8730>, <cell.Cell object at 0x1037b97e0>, <cell.Cell object at 0x1037b9a50>, <cell.Cell object at 0x106036c50>, <cell.Cell object at 0x106034190>, <cell.Cell object at 0x1060774f0>, <cell.Cell object at 0x106077370>, <cell.Cell object at 0x1060344c0>]
Cannot place black ladybug at this position because it would create separate island.
Can place piece only at cells that don't touch other player's pieces
[<cell.Cell object at 0x106035c90>, <cell.Cell object at 0x1037bb490>, <cell.Cell object at 0x1037b8730>, <cell.Cell object at 0x1037b97e0>, <cell.Cell object at 0x1037b9a50>, <cell.Cell object at 0x106036c50>, <cell.Cell object at 0x106034190>, <cell.Cell 